In [1]:
import numpy as np
import pandas as pd
import torch
import sys
import pickle

from sklearn.manifold import TSNE
import plotly.graph_objects as go

from src.params import get_params
from src.models.scale import Scale
import src.visualization.scale_color_filter as scf
import src.constants as constants

c:\Users\Proprio\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading Model and Vocabulary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load(get_params().embedding_model_path, map_location=device)

with open('./data/training_data.pkl', 'rb') as file:
        data = pickle.load(file, encoding="latin1")

vocab = data['chords_vocabulary']

c:\Users\Proprio\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:707: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


# Getting Embeddings

In [3]:
# embedding from first model layer
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms

print(f'embeddings shape: {embeddings.shape}')
print(f'embeddings norms shape: {embeddings_norm.shape}')

embeddings shape: (298, 25)
embeddings norms shape: (298, 25)


# Visualization with t-SNE

In [4]:
# get embeddings
embeddings_df = pd.DataFrame(embeddings)

# t-SNE transform
tsne = TSNE(n_components=2)
embeddings_df_trans = tsne.fit_transform(embeddings_df)
embeddings_df_trans = pd.DataFrame(embeddings_df_trans)

# get token order
chord_suffixes = np.array(list(vocab.suffixes_to_indexes.keys()))
embeddings_df_trans.index = chord_suffixes

In [5]:
colors = ['lightgrey'] * len(chord_suffixes)
scale = Scale('C', 'major')

colors = scf.get_scale_all_triads_filter(colors, chord_suffixes, scale, 'blue')
colors = scf.get_scale_chord_degress(colors, chord_suffixes, 0, 'major', 'green')
colors = scf.get_scale_main_triads_filter(colors, chord_suffixes, scale, 'red')

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=embeddings_df_trans[0],
        y=embeddings_df_trans[1],
        mode="text",
        text=embeddings_df_trans.index,
        textposition="middle center",
        textfont=dict(color=colors)
    )
)

# Similare chords

In [6]:
def get_top_similar(chord_suffix: str, topN: int = 10):
    chord_id = vocab.suffixes_to_indexes[chord_suffix]

    if chord_id == 0:
        print("Out of vocabulary chord")
        return

    chord_vec = embeddings_norm[chord_id]
    chord_vec = np.reshape(chord_vec, (len(chord_vec), 1))
    dists = np.matmul(embeddings_norm, chord_vec).flatten()
    topN_ids = np.argsort(-dists)[1 : topN + 1]

    topN_dict = {}
    for sim_chord_id in topN_ids:
        sim_chord = vocab.indexes_to_chords[sim_chord_id]
        sim_chord_suffix = ''.join(sorted(sim_chord.note_suffixes))
        topN_dict[sim_chord_suffix] = dists[sim_chord_id]

    return topN_dict

In [7]:
for chord, sim in get_top_similar("CEG").items():
    print("{}: {:.3f}".format(chord, sim))

CFG: 0.622
CEF: 0.596
BDG: 0.526
BCG: 0.485
CDG: 0.459
ACD: 0.456
ACF: 0.453
BEF: 0.450
AEG: 0.440
ACG: 0.428


# Chord vector equations

In [8]:
# C Major root chord - C Major 3rd chord + C Major 2nd chord
emb1 = embeddings[vocab.suffixes_to_indexes['CEG']] # C Major root chord 
emb2 = embeddings[vocab.suffixes_to_indexes['BEG']] # C Major 3rd chord 
emb3 = embeddings[vocab.suffixes_to_indexes['ADF']] # C Major 2nd chord

eq_emb = emb1 - emb2 + emb3
eq_emb_norm = (eq_emb ** 2).sum() ** (1 / 2)
eq_emb = eq_emb / eq_emb_norm

eq_emb = np.reshape(eq_emb, (len(eq_emb), 1))
dists = np.matmul(embeddings_norm, eq_emb).flatten()

top5 = np.argsort(-dists)[:5]

for chord_id in top5:
    chord_suffix = vocab.indexes_to_chords[chord_id]
    print("{}: {:.3f}".format(chord_suffix, dists[chord_id]))

[ A, D, F ]: 0.723
[ A, C, F ]: 0.707
[ C, D, F ]: 0.664
[ C, F, G ]: 0.609
[ A, A#, D ]: 0.594
